# 4장. 좋은 훈련 데이터셋 만들기: 데이터 전처리

* 데이터셋에서 누락된 값을 제거하거나 대체하기
* 머신 러닝 알고리즘을 위해 범주형 데이터 변환하기
* 모델과 관련이 높은 특성 선택하기

## 4.1 누락된 데이터 다루기

데이터셋에서 샘플을 제거하거나 다른 샘플이나 특성에서 누락된 값을 대체하는 기법

### 4.1.1 테이블 형태 데이터에서 누락된 값 식별


In [1]:
import pandas as pd
from io import StringIO

csv_data = \
    '''A,B,C,D
    1.0,2.0,3.0,4.0
    5.0,6.0,,8.0
    10.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [2]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

* 사이킷런에서 데이터를 적재할 때, as_frame 매개변수를 True로 지정하면 판다스 데이터프레임으로 반환
* 이제 auto로 바뀌는데 이는 데이터가 희소하지 않으면 기본적으로 판다스 데이터프레임으로 반환한다.

### 4.1.2 누락된 값이 있는 훈련 샘플이나 특성 제외

* dropna 메서드:
  * axis = 0 : NA가 있는 행 삭제
  * axis = 1 : NA가 있는 열 삭제
  * inplace = True: 새로운 데이터프레임을 반환하지 않고 주어진 데이터프레임 자체를 바꿈
  * how = 'all': 모든 열이 NaN일 때만 행을 삭제
  * thresh = 4: NaN이 아닌 값이 네 개보다 작은 행을 삭제
  * subset = ['C']: 특정 열에 NaN이 있는 행만 삭제

너무 많은 데이터를 제거하면 안정된 분석이 불가능하다.

### 4.1.3 누락된 값 대체

가장 흔한 보간 기법 중 하나는 평균으로 대체하는 것 - sklearn의 SimpleImputer 클래스  
scikit-learn의 0.24버전에서 아직 실험 중이지만, IterativeImputer 클래스로 누락된 값이 있는 한 특성을 타깃으로 삼고 다른 특성을 사용해서 모델을 훈련하여 예측  

더 쉬운 방법은 판다스의 fillna 메서드에 매개변수로 누락된 값을 채울 방법을 전달  
* df.fillna(df.mean())

fillna 메서드의 method 매개변수를 사용하여 누락된 값을 채울 수도 있음  
* df.fillna(method='bfill') # 누락된 값을 다음 행의 값으로 채움
* df.fillna(method='ffill') # 누락된 값을 이전 행의 값으로 채움
* df.fillna(method='ffill', axis=1) # 행이 아닌 열을 사용해 이전 열의 값으로 누락된 값을 채움

In [3]:
# 특성 열에서 계산한 평규능로 바꾸기
from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

### 4.1.4 사이킷런 추정기 API 익히기

변환기 클래스 = fit() + transform()  
* fit: 훈련 데이터에서 모델 파라미터를 학습
* transform: 학습한 파라미터로 데이터를 변환

추정기는 추가로 predict() 메서드가 있음

## 4.2 범주형 데이터 다루기

### 4.2.1 판다스를 사용한 범주형 데이터 인코딩


In [4]:
# 새 데이터 만들기
import pandas as pd
df = pd.DataFrame([
                    ['green', 'M', 10.1, 'class1'],
                    ['red', 'L', 13.5, 'class2'],
                    ['blue', 'XL', 15.3, 'class1']
                   ])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [5]:
size_mapping = {
                'XL':3,
                'L':2,
                'M':1
}

In [6]:
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [7]:
# 정수로 매핑된 것을 원래대로
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

### 4.2.3 클래스 레이블 인코딩

정수 배열로 전달, enumerate 사용

In [8]:
# 참고할 키:값 쌍의 딕셔너리 만들기
import numpy as np
class_mapping = {label: idx for idx, label in 
                 enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [9]:
# 정수값으로 바꾸기
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [10]:
# 다시 원본 문자열의 형태로 돌리기
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [11]:
# 사이킷런 이용하기
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

In [12]:
# 원본 문자열 형태로 되돌리기
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

### 4.2.4 순서가 없는 특성에 원-핫 인코딩 적용

일단, 앞 데이터의 color 열은 순서가 없는 범주형 데이터이므로 앞과 같은 방식으로 변경하기

In [13]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)